# Create an image classification solution with Tensorflow

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras.models import Sequential

# Setup image folder
import pathlib

data_dir = pathlib.Path('data_set')
image_paths = list(data_dir.glob('*/*.jpeg'))
print('opening first image:', image_paths[0])
PIL.Image.open(str(image_paths[0]))

create a set for training the model

In [ ]:
# Create a dataset, 
# the images will be resized automatically using the image_dataset_from_directory() utility
batch_size = 5
img_height = 180
img_width = 180

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

create a set for validating the model

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

find the classnames used

In [ ]:
class_names = train_ds.class_names
print(class_names)

See what an image looks like as a tensor:
(it is an array of 180 pixels rows with 180 pixels each (from left to right), each pixel has an RBG color value of [0-255, 0-255, 0-255])

In [ ]:
image0, class0 = list(train_ds)[0]
image0
#print(np.min(image0), np.max(image0))

Visualize/check the (resized) data using pyplot

In [ ]:
# configure plot
plt.figure(figsize=(5, 5))
for images, labels in train_ds.take(1): # take one batch of 5  
  for i in range(len(images)):
    ax = plt.subplot(3, 3, i + 1)
    imageArray = images[i].numpy().astype("uint8")    
    plt.imshow(imageArray)
    plt.title(class_names[labels[i]])
    plt.axis("off")
#show plot
plt.show()

manually iterate over the dataset and retrieve batches of images

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break # stop after first iteration


The image_batch is a tensor of the shape (5, 180, 180, 3).

This is a batch of 5 images of shape 180x180x3 (the last dimension refers to color channels RGB). 

The label_batch is a tensor of the shape (5,), these are corresponding labels to the 5 images.

The RGB channel values are in the [0, 255] range.
This is not ideal for a neural network; in general you should seek to make your input values small.

In [ ]:
normalization_layer = layers.Rescaling(1./255) # creates a funtion

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds)) #another way to get the first element of the iterable training set
first_image = image_batch[0]
first_image
# Notice the pixel values are now in `[0,1]`.
#print(np.min(first_image), np.max(first_image))


#### A basic (Keras) Sequential Model using Convolution Layers
![cnn sees](../basics/.images/how_a_cnn_sees.png)

It uses filters to make it easier for the network to recognize features, for instance by finding and highlighting edges.

![filter](../basics/.images/filters.png)

To see how filters work interactivly [click here](https://deeplizard.com/resource/pavq7noze2)

Convolution Network Layers are stacked on top of each other to reduce the information to its essentials.

```
# Great learning opportunity!
It is possible to see what a model with one Conv2D layer does to the input by examining the output when using the predict function, because the model has only one layer the output is the output of the Conv2D layer which is an array of filtered images (= an array of pixels). 

[watch video](https://pysource.com/2022/08/02/feature-map-computer-vision-with-keras-p-4/)
```

-------------
![features](../basics/.images/conv2d.png)
-------------
![features](../basics/.images/features.jpg)
-------------

Because of the feature images are filtered in each layer: Each layer will hold less neurons (=convolution) than its previous layer reducing the images to there bear essence.

In [ ]:
num_classes = len(class_names)

model = Sequential([
  # you could define the definitions of an input layer here but you don't have to
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'), # 16 different '3x3 filters' result in 16 different features being recognized.
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(), #flattens from an 2d array of [180, 180] to 1d array [32400]
  layers.Dense(128, activation='relu'), # 128 = number of neurons in a network layer, that decide if an image matches
  layers.Dense(num_classes) # the last 2 Dense layers will transform the features into output labels
])

model.summary() # view summary of the model


#### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's [*compile*](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile) step:

* [*Loss function*](https://www.tensorflow.org/api_docs/python/tf/keras/losses) —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
* [*Optimizer*](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) —This is how the model is updated based on the data it sees and its loss function.
* [*Metrics*](https://www.tensorflow.org/api_docs/python/tf/keras/metrics) —Used to monitor the training and testing steps. The following example uses *accuracy*, the fraction of the images that are correctly classified.

Choose the *tf.keras.optimizers.Adam optimizer* and *tf.keras.losses.SparseCategoricalCrossentropy* loss function. To view training and validation accuracy for each training epoch, pass the metrics argument to Model.compile.

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #Computes how often integer targets are in the top K predictions.
  metrics=['accuracy']) #monitor accuracy after each training epoch


#### Train

train the model by fitting the training data and testing the accuracy and then testing the accuracy using the validation data.

During the different of epochs (optimization runs) keep a close eay on the *increase* in accuracy:

In [ ]:
# train the model with only 3! steps
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3,
  verbose=2
)


### Visualize Training accurracy and Validation accuracy
To help you create better models make the training accuracy and validation accuracy visible in a plot.

In [ ]:
# now train it with an additional 10 steps
epochs = 10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

#### Predict using data unknown to model

In [ ]:
unclassified_image_url = "https://www.captain-hook.nl/controller/images/003394c0a45a936a58f44658e9fc05d2.jpg"
# save to keras cache
unclassified_image_path = tf.keras.utils.get_file('unclassified.jpeg', origin=unclassified_image_url)
print(unclassified_image_path)
PIL.Image.open(unclassified_image_path)

In [ ]:
img = tf.keras.utils.load_img(
    unclassified_image_path, target_size=(img_height, img_width)
)

img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

#### Improving your model

##### Overfitting
If the accuracy it start to decline or stagnate (after peaking) it means there is overfitting: 
The training data starts to fit the model too perfectly! (When there are a small number of training examples, the model sometimes learns from noises or unwanted details from training examples) While the accuracy on the validation set is not improving. Not good: what you really want is to develop models that do well on a data set they haven't seen before.

To prevent overfitting, the best solution is to use more training data with examples of what you want to predict. The dataset should cover the full range of inputs that the model is expected to handle. Additional data may only be useful if it covers new and interesting cases.

Another way to prevent overfitting is by using **data augmentation**, in our example by supplying a flipped version of the images..

##### Underfitting
The opposite of overfitting is underfitting. Underfitting occurs when there is still room for improvement on the train data. This can happen for a number of reasons: If the model is not powerful enough, is over-regularized, or has simply not been trained long enough. This means the network has not learned the relevant patterns in the training data.

In [ ]:
# Data augmentation takes the approach of generating additional training data from 
# your existing examples by augmenting them using random transformations that yield 
# believable-looking images. This helps expose the model to more aspects of the data 
# and generalize better.
# 
# uncomment lines below to add more data:
model = Sequential([
 layers.RandomFlip("horizontal",
                     input_shape=(img_height,
                                 img_width,
                                 3)),
 layers.RandomRotation(0.1),
 layers.RandomZoom(0.1),
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(1000, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])
  
# run previous step to train the model and visualize the plot again

#### Save model
Use TensorFlow Lite

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Load the tflite model in Python

In [ ]:
TF_MODEL_FILE_PATH = 'model.tflite' # The default path to the saved TensorFlow Lite model

interpreter = tf.lite.Interpreter(model_path=TF_MODEL_FILE_PATH)
# get input and output
interpreter.get_signature_list()

Use the output from the previous block to identify the runner, first input layer,  and the last output layer

i.e. {'serving_default': {'inputs': ['rescaling_3_input'], 'outputs': ['dense_7']}}

In [ ]:
runner = 'serving_default'
inputs = 'rescaling_3_input'
outputs = 'dense_7'

classify_lite = interpreter.get_signature_runner(runner)
predictions_lite = classify_lite(rescaling_3_input=img_array)[outputs] 
score_lite = tf.nn.softmax(predictions_lite)

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score_lite)], 100 * np.max(score_lite))
)
print('dif between normal and lite model prediction', np.max(np.abs(predictions - predictions_lite)))

